In [1]:
from ais import functions as af
from pyspark.sql import functions as F

In [2]:
import pandas as pd
import geopandas as gpd
import h3.api.numpy_int as h3int

In [24]:
import folium
from folium.plugins import HeatMap
from shapely.geometry import Polygon, Point, mapping

In [4]:
 #Optional -- user preferences
pd.set_option('display.max_columns', None) #Show all columns in pandas df
pd.set_option('display.max_rows', 100) #Show 100 rows in pandas df
pd.options.display.float_format = '{:,.2f}'.format

from IPython.core.interactiveshell import InteractiveShell #allow multiple outputs in one jupyter cell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
s3path = "s3a://ungp-ais-data-historical-backup/user_temp/adb/new_ports_canals/"

# Suez

In [23]:
n = spark.read.parquet(s3path+"Suez/point_agg/").count()
df = spark.read.parquet(s3path+"Suez/point_agg/").sample(fraction = 10000/n).toPandas()

In [10]:
geom = gpd.GeoDataFrame(pd.concat([pd.read_pickle(s3path+"Suez/North_geometry.pkl").assign(label="North"),
                  pd.read_pickle(s3path+"Suez/South_geometry.pkl").assign(label="South")]
                ), geometry="geometry", crs="epsg:4326")
geom

,geometry,labels,h3,h3_bound,label
0,"POLYGON ((32.31540 30.89807, 32.31452 30.89844...",0,"[618090828786040831, 618090826765697023, 61809...","POLYGON ((32.19062018899972 31.38242837291015,...",North
0,"POLYGON ((32.46171 29.89680, 32.46361 29.91027...",0,"[618090673989746687, 618090705293934591, 61809...",POLYGON ((32.540345047340246 29.74987011984434...,South


In [27]:
lat, long = 31.26667,32.30000

# non-AOI
gdf2 = gpd.GeoDataFrame([[Point( 32.301075640052424, 31.65971930969196)], [Point( 32.568732975340495, 29.609886500955657)]],columns=['geometry'], crs='epsg:4326')
gdf2 = gdf2.to_crs("epsg:22991")
gdf2['buffer'] = gdf2.buffer(50000, cap_style=3)
gdf2.set_geometry('buffer', inplace=True)
gdf2.to_crs("epsg:4326", inplace=True)


m = folium.Map(location=[lat,long],zoom_start=10, tiles='cartodbpositron')
a = folium.Marker(location = [lat,long]).add_to(m)   
a = HeatMap(df[['latitude','longitude','count_ais']].values, 
                radius=2,
                blur=2,
                name="HeatMap").add_to(m)  

a = folium.GeoJson(gdf2[['buffer']].to_json(), 
                   style_function = lambda x: {'weight':2,'fillOpacity':0.5,'color':'gray'},
                   show=False).add_to(m)
a = folium.GeoJson(geom[['geometry']].to_json(), 
                   style_function = lambda x: {'weight':2,'fillOpacity':0.5,'color':'gray'},
                   show=False).add_to(m)
# a = folium.LayerControl().add_to(m)

In [28]:
m

In [30]:
gdf2['label'] = ["North","South"]

In [32]:
gdf2.to_pickle(s3path+"Suez/NonAOI_geometry.pkl")

# Panama Canal

In [33]:
n = spark.read.parquet(s3path+"Panama/point_agg/").count()
df = spark.read.parquet(s3path+"Panama/point_agg/").sample(fraction = 10000/n).toPandas()

In [35]:
geom = gpd.GeoDataFrame(pd.concat([pd.read_pickle(s3path+"Panama/North_geometry.pkl").assign(label="North"),
                  pd.read_pickle(s3path+"Panama/South_geometry.pkl").assign(label="South")]
                ), geometry="geometry", crs="epsg:4326")
geom

,geometry,labels,h3,h3_bound,label
0,"POLYGON ((-80.01238 9.45625, -80.01357 9.46251...",0,"[618798943064817663, 618798942314561535, 61879...",POLYGON ((-79.88182255464089 9.201328674637681...,North
0,"POLYGON ((-79.59114 8.80343, -79.59244 8.80404...",0,"[618798906415513599, 618798891935203327, 61879...","POLYGON ((-79.478363789723 8.833225728404186, ...",South


In [69]:
gdf = gpd.GeoDataFrame([[Point( -79.90820699820482, 9.48844708804138,)], [Point(-79.55692544545876, 8.760721138565856)],
                       [Point( -79.34833647472335, 8.899762522149963)]],columns=['geometry'], crs='epsg:4326')
gdf = gdf.to_crs("epsg:2385")
gdf['buffer'] = gdf.buffer(20000, cap_style=3)
gdf.set_geometry('buffer', inplace=True)
gdf.to_crs("epsg:4326", inplace=True)

In [70]:
lat, long = 9.386487, -79.919638

m = folium.Map(location=[lat,long],zoom_start=10, tiles='cartodbpositron')
a = folium.Marker(location = [lat,long]).add_to(m)
a = HeatMap(df[['latitude','longitude','count_ais']].values, 
                radius=2,
                blur=2,
                name="HeatMap").add_to(m)   

a = folium.GeoJson(gdf[['buffer']].to_json(), 
                   style_function = lambda x: {'weight':2,'fillOpacity':0.5,'color':'gray'},
                   show=False).add_to(m)

a = folium.GeoJson(geom[['geometry']].to_json(), 
                   style_function = lambda x: {'weight':2,'fillOpacity':0.5,'color':'gray'},
                   show=False).add_to(m)


a = folium.GeoJson(gdf['buffer']).add_to(m)
m

In [72]:
gdf['label'] = ["North","South","South"]

In [75]:
gdf.to_pickle(s3path+"Panama/NonAOI_geometry.pkl")